In [2]:
import ollama
import pandas as pd

## Classification Schema Selection

Selection:

binary1: irrelevant vs. explicit/implicit/relchanges/releconomy

binary2: relchanges/releconomy/irrelevant vs. explicit/implicit

binary3: releconomy/irrelevant vs. explicit/implicit/relchanges

three_code1: irrelevant vs. implicit/relchanges/releconomy vs. explicit

three_code2: irrelevant vs. relchanges/releconomy vs. explicit/implicit

three_code3: releconomy/irrelevant vs. relchanges vs. explicit/implicit

four_code: irrelevant vs. relchanges/releconomy	 vs. explicit vs. implicit

four_code2: irrelevant vs. relchanges vs. explicit/implicit vs. releconomy

five_code: irrelevant vs. relchanges vs. explicit vs. releconomy vs. implicit
    

In [4]:
corpus = pd.read_json('../data/old_data_clean.json')

def map_values(value):
    if value == 'irrelevant':
        return '5'
    elif value == 'relchanges':
        return '3'
    elif value == 'explicit':
        return '1'
    elif value == 'releconomy':
        return '4'
    elif value == 'implicit':
        return '2'
    else:
        return 'Unknown'

# Apply the function to create the new column
corpus['label'] = corpus['five_code'].apply(map_values)
rel_corpus = corpus[corpus['label'] != '5'].reset_index(drop=True)

In [ ]:
max_char = 8000 * 4

output_list = []

meta_statement = """We categorize articles that are related to issues of income inequality, changes in income or wealth, general economic conditions."""

fewshot_prompt = """"  Does the article reference American economic inequality? Respond with 'Yes' if article meets any or all criteria referencing American economic inequality and 'No' if article meets none of the criteria, and explain why in 1 sentence. Input: "Blame Game An unalterable truth in Washington is that everything is about the next election--as President Bush is about to find out in the upcoming battle of the budget. In the Clinton era, budget wars were rollicking political brawls with defining consequences. The best and most obvious example: the government shutdown of 1995. With the economy lagging, the surplus evaporating, and the Democrats controlling the Senate, President Bush enters his first budget fight with fewer advantages than he held on Inauguration Day. Soon he must confront the reality that while the budget deals with the nitty-gritty of collecting and spending some $ 2 trillion annually, it also provides a grand stage for unsubtle overtures to voters, who go to the polls next year to elect all 435 members of the House and more than a third of the Senate. With three prominent GOP senators announcing their retirement from the closely divided Senate, Republicans have 20 seats to defend, while Democrats have only 14 incumbents up for re-election. With the planned departures of Strom Thurmond, Jesse Helms, and Phil Gramm, the Republican task of retaking the Senate grows more daunting by the day. A Democrat-controlled Senate in 2002 means that President Bush will go into his own re-election two years later with South Dakota Democrat Tom Daschle as majority leader, one who could derail the White House's agenda. Reveling in a sooner than expected bout of I-told-you-so, Democrats point to the dwindling federal budget surplus and warn that the Bush administration will be forced to break its campaign pledge that it would not use Social Security taxes for anything other than Social Security benefits. Daschle has been adroitly laying the foundation for blame. "Certainly, they are the architects of our current economic condition, and I think we have to turn to them for leadership now as we find our way out of this box that has been created," he says. The pressure points are obvious. Bush has requested an additional $ 18.3 billion for defense. There has been little progress on how to fund the president's educational reform package. His energy plan is unresolved. On the other side, Democrats want to increase the minimum wage, which most in the GOP oppose if there aren't tax breaks for small businesses. And the Democrats want a prescription drug benefit for Medicare recipients, which could cost $ 300 billion over the next decade. Bush defends his budget by saying that it could accommodate his cherished tax cut and other proposals if only Congress could restrain its chronic tendencies toward spending. But those wise to Washington's ways also know that spending has its virtues. After all, many of the millions spent on so-called pet projects will figure prominently in the political stump speeches during the next election cycle." Output: Yes
    Input: "State and local tax collections reached $320.2 billion in 1984, an increase of 12.5 percent over the previous year, the Commerce Department reported. The state-and-local tax bite averaged $1,356 per person -- or $117 for each $1,000 of personal income. Alaska led the nation with $4,704 per person and $286 per $1,000 of income. Per Per $1,000 capita income Alaska $4,704 $286 Wyoming $2,504 $209 Dist. of Columbia $2,300 $146 New York $2,130 $165 Minnesota $1,706 $144 Connecticut $1,656 $112 New Jersey $1,637 $117 Michigan $1,575 $138 Wisconsin $1,556 $137 Massachusetts $1,549 $117 Hawaii $1,543 $129 Maryland $1,503 $117 California $1,503 $115 Washington $1,416 $118 Illinois $1,405 $113 Rhode Island $1,403 $121 Delaware $1,400 $112 U.S. average $1,356 $117 Nevada $1,353 $111 Colorado $1,339 $106 North Dakota $1,334 $115 Oregon $1,321 $124 Pennsylvania $1,309 $114 Montana $1,275 $129 Iowa $1,273 $119 Vermont $1,271 $129 Kansas $1,260 $103 Arizona $1,246 $120 Ohio $1,246 $111 Nebraska $1,232 $110 Maine $1,229 $126 Virginia $1,210 $101 New Mexico $1,194 $126 Oklahoma $1,159 $106 Utah $1,133 $129 Texas $1,115 $ 97 Louisiana $1,114 $109 West Virginia $1,113 $121 Indiana $1,093 $105 New Hampshire $1,092 $ 93 Georgia $1,073 $105 Florida $1,073 $ 95 North Carolina $1,027 $106 Missouri $1,012 $ 93 South Carolina $ 981 $108 South Dakota $ 978 $100 Kentucky $ 955 $102 Idaho $ 953 $101 Alabama $ 916 $100 Tennessee $ 878 $ 93 Mississippi $ 871 $108 Arkansas $ 866 $ 97" Output: No"""

for num in range(len(rel_corpus)):

    text = 'Title: ' + rel_corpus.loc[num,'title'] + '\n' + rel_corpus.loc[num,'text'][:max_char]

    print(num)
    print(text[:20])

    content_parse = "Read this article: " + text + fewshot_prompt

    response = ollama.chat(
        model="llama3:70b",
        messages=[
            {
                "role": "system",
                "content": "You are a news classifier."
            },

            {
                "role": "user",
                "content": meta_statement,
            },

            {
                "role": "user",
                "content": content_parse
            },
        ],

        options={
        "seed": 101,
        "temperature": 0,
        "num_ctx": 8200,
        "num_thread": 20,
        #"num_predict": 10,
            }
    )
    output_list.append((response["message"]["content"]))

In [ ]:
rel_corpus['content'] = output_list

rel_corpus['gen_code'] = rel_corpus['content']
rel_corpus.loc[rel_corpus['content'].str.lower().str.startswith('no'), 'gen_code'] = 0
rel_corpus.loc[rel_corpus['content'].str.lower().str.startswith('yes'), 'gen_code'] = 1

rel_corpus.loc[rel_corpus['content'].str.lower().str.contains('output: yes'), 'gen_code'] = 1
rel_corpus.loc[rel_corpus['content'].str.lower().str.contains('output: no'), 'gen_code'] = 0

rel_corpus.loc[rel_corpus['content'].str.lower().str.startswith('the output for the first article is "yes"'), 'gen_code'] = 1
rel_corpus.loc[rel_corpus['content'].str.lower().str.startswith('the output for the first article is "no"'), 'gen_code'] = 0

rel_corpus.loc[rel_corpus['gen_code'].str.len()>3, 'gen_code'] = 0

columns = ['title', 'year', 'month', 'journal', 'code', 'weight', 'code_label', 'id', 'code', 'content', 'gen_code']
rel_corpus = rel_corpus[columns]

In [24]:
rel_corpus.to_csv('../data/inequality_dataset_llama3-70b-Inequality-FewShot-NoDefinition.csv')